In [1]:
import io
import pyaudio

In [2]:
from snowboy.snowboydetect import SnowboyDetect

In [3]:
import wave
import audioop
import collections
import math
import sys
import os
import time

# Source

In [4]:
audio = pyaudio.PyAudio()

In [5]:
count = audio.get_device_count()

Use device at position zero.

In [6]:
device_index = 0

In [7]:
device_info = audio.get_device_info_by_index(device_index)
sample_rate = int(device_info["defaultSampleRate"])

### Settings source

In [8]:
form = pyaudio.paInt16
SAMPLE_WIDTH = pyaudio.get_sample_size(form)
SAMPLE_RATE = sample_rate
CHUNK = 1024

### Microphone stream

In [9]:
class MicrophoneStream(object):
    def __init__(self, pyaudio_stream):
        self.pyaudio_stream = pyaudio_stream

    def read(self, size):
        return self.pyaudio_stream.read(size, exception_on_overflow=False)

    def close(self):
        try:
            # sometimes, if the stream isn't stopped, closing the stream throws an exception
            if not self.pyaudio_stream.is_stopped():
                self.pyaudio_stream.stop_stream()
        finally:
            self.pyaudio_stream.close()

In [10]:
stream = MicrophoneStream(
    audio.open(
        input_device_index=device_index, channels=1, format=form,
        rate=SAMPLE_RATE, frames_per_buffer=CHUNK, input=True,
    )
)

# Audio data output

In [11]:
class AudioData(object):
    """
    Creates a new ``AudioData`` instance, which represents mono audio data.
    The raw audio data is specified by ``frame_data``, which is a sequence of bytes representing audio samples. This is the frame data structure used by the PCM WAV format.
    The width of each sample, in bytes, is specified by ``sample_width``. Each group of ``sample_width`` bytes represents a single audio sample.
    The audio data is assumed to have a sample rate of ``sample_rate`` samples per second (Hertz).
    Usually, instances of this class are obtained from ``recognizer_instance.record`` or ``recognizer_instance.listen``, or in the callback for ``recognizer_instance.listen_in_background``, rather than instantiating them directly.
    """
    def __init__(self, frame_data, sample_rate, sample_width):
        assert sample_rate > 0, "Sample rate must be a positive integer"
        assert sample_width % 1 == 0 and 1 <= sample_width <= 4, "Sample width must be between 1 and 4 inclusive"
        self.frame_data = frame_data
        self.sample_rate = sample_rate
        self.sample_width = int(sample_width)

    def get_segment(self, start_ms=None, end_ms=None):
        """
        Returns a new ``AudioData`` instance, trimmed to a given time interval. In other words, an ``AudioData`` instance with the same audio data except starting at ``start_ms`` milliseconds in and ending ``end_ms`` milliseconds in.
        If not specified, ``start_ms`` defaults to the beginning of the audio, and ``end_ms`` defaults to the end.
        """
        assert start_ms is None or start_ms >= 0, "``start_ms`` must be a non-negative number"
        assert end_ms is None or end_ms >= (0 if start_ms is None else start_ms), "``end_ms`` must be a non-negative number greater or equal to ``start_ms``"
        if start_ms is None:
            start_byte = 0
        else:
            start_byte = int((start_ms * self.sample_rate * self.sample_width) // 1000)
        if end_ms is None:
            end_byte = len(self.frame_data)
        else:
            end_byte = int((end_ms * self.sample_rate * self.sample_width) // 1000)
        return AudioData(self.frame_data[start_byte:end_byte], self.sample_rate, self.sample_width)

    def get_raw_data(self, convert_rate=None, convert_width=None):
        """
        Returns a byte string representing the raw frame data for the audio represented by the ``AudioData`` instance.
        If ``convert_rate`` is specified and the audio sample rate is not ``convert_rate`` Hz, the resulting audio is resampled to match.
        If ``convert_width`` is specified and the audio samples are not ``convert_width`` bytes each, the resulting audio is converted to match.
        Writing these bytes directly to a file results in a valid `RAW/PCM audio file <https://en.wikipedia.org/wiki/Raw_audio_format>`__.
        """
        assert convert_rate is None or convert_rate > 0, "Sample rate to convert to must be a positive integer"
        assert convert_width is None or (convert_width % 1 == 0 and 1 <= convert_width <= 4), "Sample width to convert to must be between 1 and 4 inclusive"

        raw_data = self.frame_data

        # make sure unsigned 8-bit audio (which uses unsigned samples) is handled like higher sample width audio (which uses signed samples)
        if self.sample_width == 1:
            raw_data = audioop.bias(raw_data, 1, -128)  # subtract 128 from every sample to make them act like signed samples

        # resample audio at the desired rate if specified
        if convert_rate is not None and self.sample_rate != convert_rate:
            raw_data, _ = audioop.ratecv(raw_data, self.sample_width, 1, self.sample_rate, convert_rate, None)

        # convert samples to desired sample width if specified
        if convert_width is not None and self.sample_width != convert_width:
            if convert_width == 3:  # we're converting the audio into 24-bit (workaround for https://bugs.python.org/issue12866)
                raw_data = audioop.lin2lin(raw_data, self.sample_width, 4)  # convert audio into 32-bit first, which is always supported
                try: audioop.bias(b"", 3, 0)  # test whether 24-bit audio is supported (for example, ``audioop`` in Python 3.3 and below don't support sample width 3, while Python 3.4+ do)
                except audioop.error:  # this version of audioop doesn't support 24-bit audio (probably Python 3.3 or less)
                    raw_data = b"".join(raw_data[i + 1:i + 4] for i in range(0, len(raw_data), 4))  # since we're in little endian, we discard the first byte from each 32-bit sample to get a 24-bit sample
                else:  # 24-bit audio fully supported, we don't need to shim anything
                    raw_data = audioop.lin2lin(raw_data, self.sample_width, convert_width)
            else:
                raw_data = audioop.lin2lin(raw_data, self.sample_width, convert_width)

        # if the output is 8-bit audio with unsigned samples, convert the samples we've been treating as signed to unsigned again
        if convert_width == 1:
            raw_data = audioop.bias(raw_data, 1, 128)  # add 128 to every sample to make them act like unsigned samples again

        return raw_data

    def get_wav_data(self, convert_rate=None, convert_width=None):
        """
        Returns a byte string representing the contents of a WAV file containing the audio represented by the ``AudioData`` instance.
        If ``convert_width`` is specified and the audio samples are not ``convert_width`` bytes each, the resulting audio is converted to match.
        If ``convert_rate`` is specified and the audio sample rate is not ``convert_rate`` Hz, the resulting audio is resampled to match.
        Writing these bytes directly to a file results in a valid `WAV file <https://en.wikipedia.org/wiki/WAV>`__.
        """
        raw_data = self.get_raw_data(convert_rate, convert_width)
        sample_rate = self.sample_rate if convert_rate is None else convert_rate
        sample_width = self.sample_width if convert_width is None else convert_width

        # generate the WAV file contents
        with io.BytesIO() as wav_file:
            wav_writer = wave.open(wav_file, "wb")
            try:  # note that we can't use context manager, since that was only added in Python 3.4
                wav_writer.setframerate(sample_rate)
                wav_writer.setsampwidth(sample_width)
                wav_writer.setnchannels(1)
                wav_writer.writeframes(raw_data)
                wav_data = wav_file.getvalue()
            finally:  # make sure resources are cleaned up
                wav_writer.close()
        return wav_data

    def get_aiff_data(self, convert_rate=None, convert_width=None):
        """
        Returns a byte string representing the contents of an AIFF-C file containing the audio represented by the ``AudioData`` instance.
        If ``convert_width`` is specified and the audio samples are not ``convert_width`` bytes each, the resulting audio is converted to match.
        If ``convert_rate`` is specified and the audio sample rate is not ``convert_rate`` Hz, the resulting audio is resampled to match.
        Writing these bytes directly to a file results in a valid `AIFF-C file <https://en.wikipedia.org/wiki/Audio_Interchange_File_Format>`__.
        """
        raw_data = self.get_raw_data(convert_rate, convert_width)
        sample_rate = self.sample_rate if convert_rate is None else convert_rate
        sample_width = self.sample_width if convert_width is None else convert_width

        # the AIFF format is big-endian, so we need to covnert the little-endian raw data to big-endian
        if hasattr(audioop, "byteswap"):  # ``audioop.byteswap`` was only added in Python 3.4
            raw_data = audioop.byteswap(raw_data, sample_width)
        else:  # manually reverse the bytes of each sample, which is slower but works well enough as a fallback
            raw_data = raw_data[sample_width - 1::-1] + b"".join(raw_data[i + sample_width:i:-1] for i in range(sample_width - 1, len(raw_data), sample_width))

        # generate the AIFF-C file contents
        with io.BytesIO() as aiff_file:
            aiff_writer = aifc.open(aiff_file, "wb")
            try:  # note that we can't use context manager, since that was only added in Python 3.4
                aiff_writer.setframerate(sample_rate)
                aiff_writer.setsampwidth(sample_width)
                aiff_writer.setnchannels(1)
                aiff_writer.writeframes(raw_data)
                aiff_data = aiff_file.getvalue()
            finally:  # make sure resources are cleaned up
                aiff_writer.close()
        return aiff_data

# Recogniser

In [12]:
energy_threshold = 300
dynamic_energy_treshold = True
dynamic_energy_adjustment_damping = 0.15
dynamic_energy_ratio = 1.5
pause_threshold = 0.8  # seconds of non-speaking audio before a phrase is considered complete
operation_timeout = None  # seconds after an internal operation (e.g., an API request) starts before it times out, or ``None`` for no timeout

phrase_threshold = 1  # minimum seconds of speaking audio before we consider the speaking audio a phrase - values below this are ignored (for filtering out clicks and pops)
non_speaking_duration = 0.5  # seconds of non-speaking audio to keep on both sides of the recording

### Adjust ambient noise dynamically

In [13]:
def adjust_for_ambient_noise(energy_threshold,duration=1):

    seconds_per_buffer = (CHUNK + 0.0) / SAMPLE_RATE
    elapsed_time = 0

    # adjust energy threshold until a phrase starts
    while True:
        elapsed_time += seconds_per_buffer
        if elapsed_time > duration: break
        buffer = stream.read(CHUNK)
        energy = audioop.rms(buffer, SAMPLE_WIDTH)  # energy of the audio signal

        # dynamically adjust the energy threshold using asymmetric weighted average
        damping = dynamic_energy_adjustment_damping ** seconds_per_buffer  # account for different chunk sizes and rates
        target_energy = energy * dynamic_energy_ratio
        energy_threshold = energy_threshold * damping + target_energy * (1 - damping)
        return energy_threshold

In [14]:
energy_threshold

300

In [15]:
energy_threshold = adjust_for_ambient_noise(energy_threshold,5)

In [16]:
energy_threshold

335.74720646201945

## Snowboy

In [ ]:
"snowboy/resources/models/snowboy.umdl"

In [17]:
from file import *
TOP_DIR = get_absolute_path()

In [43]:
RESOURCE_FILE = os.path.join(TOP_DIR, "snowboy/resources/common.res")
model_str = ",".join(["snowboy/resources/models/computer.umdl"])

detector = SnowboyDetect(RESOURCE_FILE.encode(), model_str=model_str.encode())

detector.SetAudioGain(1.0)
detector.SetSensitivity(",".join(["0.6"]).encode())
detector.ApplyFrontend(True)

In [44]:
def snowboy_hot_word(detector, CHUNK, SAMPLE_RATE,SAMPLE_WIDTH, stream):
    snowboy_sample_rate = detector.SampleRate()
    seconds_per_buffer = CHUNK/SAMPLE_RATE
    resampling_state = None
    
    five_seconds_buffer_count = int(math.ceil(2/seconds_per_buffer))
    half_second_buffer_count = int(math.ceil(0.5/seconds_per_buffer))
    
    frames = collections.deque(maxlen=five_seconds_buffer_count)
    resampled_frames = collections.deque(maxlen=half_second_buffer_count)
    
    check_interval = 0.05
    last_check = time.time()
    while True:
        #elapsed_time += seconds_per_buffer
        
        buffer = stream.read(CHUNK)
        frames.append(buffer)
        
        resampled_buffer, resampling_state = audioop.ratecv(buffer, SAMPLE_WIDTH, 1, SAMPLE_RATE, snowboy_sample_rate, resampling_state)
        resampled_frames.append(resampled_buffer)
        
        if time.time() - last_check > check_interval:
            snowboy_result = detector.RunDetection(b"".join(resampled_frames))
            #print(len(resampled_frames))
            if snowboy_result == 0:
                #print('kk')
                print(len(resampled_frames))
            if snowboy_result > 0:
                print(snowboy_result)
                break;
            resampled_frames.clear()
            last_check = time.time()
            
    return b"".join(frames)

## Listen

In [45]:
def listen(stream, CHUNK, SAMPLE_RATE, pause_threshold, phrase_threshold, non_speaking_duration,energy_threshold, detector=detector):
    seconds_per_buffer = float(CHUNK)/SAMPLE_RATE
    pause_buffer_count = int(math.ceil(pause_threshold / seconds_per_buffer))
    phrase_buffer_count = int(math.ceil(phrase_threshold / seconds_per_buffer))
    non_speaking_buffer_count = int(math.ceil(non_speaking_duration / seconds_per_buffer))
    
    elapsed_time = 0
    buffer = b""
    while True:
        frames = collections.deque()
        
        if detector is None:
            while True:
                buffer = stream.read(CHUNK)
                frames.append(buffer)
                if len(frames) > non_speaking_buffer_count:
                    frames.popleft()
                energy = audioop.rms(buffer, SAMPLE_WIDTH)
                if energy > energy_threshold:
                    print('hi')
                    break;
        else:
            buffer = snowboy_hot_word(detector, CHUNK, SAMPLE_RATE,SAMPLE_WIDTH, stream)
            frames.append(buffer)
            
        pause_count, phrase_count = 0,0
        while True:
            buffer = stream.read(CHUNK)
            frames.append(buffer)
            phrase_count += 1

            energy = audioop.rms(buffer, SAMPLE_WIDTH)
            if energy > energy_threshold:
                print('hey')
                pause_count = 0
            else:
                pause_count += 1

            if pause_count > pause_buffer_count:
                break;

        phrase_count -= pause_count
        if phrase_count >= phrase_buffer_count:
            break;
            
    for i in range(pause_count - non_speaking_buffer_count):
        frames.pop()
        
    frame_data = b"".join(frames)
    
    return(AudioData(frame_data, SAMPLE_RATE, SAMPLE_WIDTH))

In [46]:
a = listen(stream, CHUNK, SAMPLE_RATE, pause_threshold, phrase_threshold, non_speaking_duration,energy_threshold)

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey


In [39]:
a

In [40]:
wv = a.get_wav_data()
with open('bla.wav', 'wb') as output:
    output.write(wv)

In [20]:
wv

b'RIFF$0\x03\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00D\xac\x00\x00\x88X\x01\x00\x02\x00\x10\x00data\x000\x03\x00\x03\x00\x01\x00\x02\x00\x00\x00\xfc\xff\xfd\xff\xfa\xff\xf3\xff\x00\x00\xff\xff\xf7\xff\x01\x00\x03\x00\x03\x00\n\x00\t\x00\x0b\x00\x06\x00\x05\x00\x08\x00\x07\x00\x04\x00\x00\x00\x06\x00\x04\x00\x04\x00\x04\x00\x04\x00\n\x00\t\x00\x07\x00\x06\x00\x08\x00\t\x00\t\x00\r\x00\x0b\x00\x0c\x00\x0e\x00\x0c\x00\x0b\x00\x06\x00\n\x00\x0e\x00\x0b\x00\x0b\x00\x11\x00\x13\x00\x0e\x00\r\x00\x08\x00\x01\x00\x03\x00\x02\x00\x04\x00\x02\x00\xfe\xff\x05\x00\x01\x00\x00\x00\x05\x00\x01\x00\x08\x00\x08\x00\x07\x00\x0b\x00\x04\x00\x08\x00\x10\x00\n\x00\x0e\x00\x10\x00\x0f\x00\x11\x00\r\x00\x07\x00\x07\x00\t\x00\x06\x00\x03\x00\x02\x00\x00\x00\xfd\xff\xfd\xff\x02\x00\x04\x00\xff\xff\xfe\xff\x00\x00\xfd\xff\xfe\xff\x05\x00\x00\x00\x00\x00\x07\x00\t\x00\x0c\x00\x07\x00\x07\x00\t\x00\n\x00\x05\x00\x01\x00\xff\xff\xfb\xff\xfd\xff\xfc\xff\xfd\xff\x05\x00\x01\x00\x05\x00\x0c\x00\x06\x00\x04\x00\x0